In [ ]:
import pandas as pd
import numpy as np

In [ ]:
estados = pd.read_csv('brasil_estados.csv')
estados

In [ ]:
round(estados['area'].describe(),2)

In [ ]:
area = estados[['uf', 'area']]
area

In [ ]:
media = area['area'].mean()
dp = area['area'].std()
area['zscore'] = area['area'].apply(lambda x : (x - media) / dp)
round(area,4)

In [ ]:
colunas = ['area', 'populacao', 'idh']
sel = colunas
sel.insert(0, "uf")
estadosz = estados[sel].copy()
listaz = []
for coluna in colunas:
    if coluna != "uf":
        print(coluna)
        listaz.insert(0, 'z_'+coluna)
        media = estadosz[coluna].mean()
        dp = estadosz[coluna].std()
        estadosz['z_'+coluna] = estadosz[coluna].apply(lambda x : (x - media) / dp)
estadosz['totalz'] = estadosz[listaz].sum(axis=1)
estadosz['ranking'] = estadosz['totalz'].rank(ascending=False)
estadosz    

In [ ]:
listaz

In [ ]:
estadosz['totalz'] = estadosz[listaz].sum(axis=1)
estadosz['ranking'] = estadosz['totalz'].rank(ascending=False)
estadosz